# Add LAMMPS Potentials

This Notebook provides details and demonstrations about how PotentialLAMMPS objects can be easily generated for potentials not currently in the database.  This allows for users to easily integrate their personal LAMMPS potentials with the database.

In [1]:
import potentials

## 1. Defining new LAMMPS potentials

For the majority of LAMMPS pair_styles, the build_potential_LAMMPS function can be used to easily generate a PotentialLAMMPS object. The function is a wrapper around numerous subclasses of potentials.build.PotentialLAMMPSBuilder, each of which is designed to properly generate the pair_coeff lines for a given set of pair_styles. There are hundreds of LAMMPS pair_styles, and each pair_style is expecting pair_coeff lines specific to that style. Luckily, the majority of pair_styles have pair_coeff lines that fall into one of two basic formats, and others are known slight variations of that. Thus, a handful of classes can properly capture the vast majority of LAMMPS pair_styles.

__General build_potential_LAMMPS() parameters associated with generating LAMMPS inputs__

- __pair_style__ (*str*) The LAMMPS pair_style to use.

- __pair_style_terms__ (*list, optional*) Any other terms that appear on the pair_style line (like cutoff) if needed.

- __symbols__ (*list, optional*) The symbols used to identify each unique particle model. Optional if elements is given and the particle symbols are the same as the elemental symbols.

- __elements__ (*list, optional*) The elemental symbols associated with each particle model if the particles represent atoms.

- __masses__ (*list, optional*) The masses of each particle.  Optional if elements is given as standard values can be used.

- __charges__ (*list, optional*) The static charges to assign to each particle, if the model calls for it.

- __allsymbols__ (*bool, optional*) Flag indicating if the coefficient lines must be defined for every particle model in the potential even if those particles are not used.  Default value is False as most pair_styles do not require this.

- __command_terms__ (*list, optional*) Allows any other LAMMPS command lines that must be set for the potential to work properly to be set.  Each command line should be given as a list of terms, and multiple command lines given as a list of lists.

- __units__ (*str, optional*) The LAMMPS units setting to use with the potential, if specific units are required. 

- __atom_style__ (*str, optional*) The LAMMPS atom_style setting to use, if a specific atom_style is required.

Any of the general parameters and pair_style-specific parameters can be directly set as object attributes any time after the object has been created.

### 1.1 Pair potentials

This is the most common format. It is used by all potentials that do not require reading parameter files.

    pair_coeff 1 1 <a_11> <b_11> ...
    pair_coeff 1 2 <a_12> <b_12> ...
    ...

__Format-specific build_potential_LAMMPS() parameters__

- __interactions__ (*list or dict*) Specifies the *symbols* and *terms* associated with each unique pair_coeff interaction.

The associated class also has the set_interaction() method that makes defining the interactions slightly more intuitive.

In [8]:
# Define pair potential with elements Al and Cu
b = potentials.build_potential_LAMMPS(pair_style='lj/cut', elements=['Al', 'Cu'])

# Specify coefficients for each unique interaction (Note: values are junk NOT REAL POTENTIALS!)
b.set_interaction(symbols=['Al', 'Al'], terms=[1.23, 3.412])
b.set_interaction(symbols=['Cu', 'Al'], terms=[1.124, 2.124])
b.set_interaction(symbols=['Cu', 'Cu'], terms=[5.324, 3.14])

# Have the builder return a PotentialLAMMPS object
pot = b.potential()
print(pot.pair_info())

mass 1 26.9815385
mass 2 63.546

pair_style lj/cut
pair_coeff 1 1 1.23 3.412
pair_coeff 1 2 1.124 2.124
pair_coeff 2 2 5.324 3.14



### 1.2 Parameter file potentials

This is the second most common format.  It is used by nearly all potentials that read in a single parameter file and associate atom types to model symbols.

    pair_coeff * * <paramfile> <list of model symbols>

__Format-specific build_potential_LAMMPS() parameters__

- __paramfile__ (*str*) The name of the potential parameter file.

In [9]:
# Define potential for Al.eam.alloy
b = potentials.build_potential_LAMMPS(pair_style='eam/alloy', elements='Al', paramfile='Al.eam.alloy')

# Have the builder return a PotentialLAMMPS object
pot = b.potential()
print(pot.pair_info())

mass 1 26.9815385

pair_style eam/alloy
pair_coeff * * Al.eam.alloy Al



### 1.3 Original EAM pair_style potentials

The original eam pair_style uses a unique format due to it defining elemental models in parameter files and using a universal mixing function.  

    pair_coeff 1 1 <paramfiles[0]>
    pair_coeff 2 2 <paramfiles[1]>
    ...

__Format-specific build_potential_LAMMPS() parameters__

- __paramfiles__ (*list*) The names of the elemental potential parameter files.

In [10]:
# Define potential using the Al.eam and Cu.eam parameter files
b = potentials.build_potential_LAMMPS(pair_style='eam', elements=['Al', 'Cu'], paramfiles=['Al.eam', 'Cu.eam'])

# Have the builder return a PotentialLAMMPS object
pot = b.potential()
print(pot.pair_info())

mass 1 26.9815385
mass 2 63.546

pair_style eam
pair_coeff 1 1 Al.eam
pair_coeff 2 2 Cu.eam



### 1.4 MEAM potentials

MEAM potentials use two types of parameter files: a general library file and a model-specific parameter file.

    pair_coeff * * <libfile> <list of libfile symbols> <paramfile> <list of model symbols>

__Format-specific build_potential_LAMMPS() parameters__

- __libfile__ (*str*) The name of the potential library file.

- __paramfile__ (*str, optional*) The name of the potential parameter file. Might not be used by older MEAM potentials.

*Note*: The MEAM parameter files for alloy (multi-element) models need to know which particle models to associate their parameters with.  As such, the first list of symbols in the pair_coeff line must be in a specific order.  Here, the builder will use the order of symbols/elements parameters as given.

In [11]:
# Define MEAM potential using the lib.meam and Al.meam parameter file
b = potentials.build_potential_LAMMPS(pair_style='meam', elements='Al', libfile='lib.meam', paramfile='Al.meam')

pot = b.potential()
print(pot.pair_info())

mass 1 26.9815385

pair_style meam
pair_coeff * * lib.meam Al Al.meam Al



In [12]:
# Define MEAM potential using the lib.meam only
b = potentials.build_potential_LAMMPS(pair_style='meam', elements='Al', libfile='lib.meam')

# Have the builder return a PotentialLAMMPS object
pot = b.potential()
print(pot.pair_info())

mass 1 26.9815385

pair_style meam
pair_coeff * * lib.meam Al NULL Al



### 1.5 EIM potentials

The eim pair_style uses a slight variation on the parameter file and MEAM formats

    pair_coeff * * <list of libfile symbols> <parameter_file> <list of model symbols>

__Format-specific build_potential_LAMMPS() parameters__

- __paramfile__ (*str*) The name of the potential parameter file.

In [13]:
b = potentials.build_potential_LAMMPS(pair_style='eim', elements=['Al', 'Cu'], paramfile='AlCu.eim')

pot = b.potential()
print(pot.pair_info())

mass 1 26.9815385
mass 2 63.546

pair_style eim
pair_coeff * * Al Cu AlCu.eim Al Cu



### 1.6 KIM potentials

This is used with the older pair_style kim not the newer kim LAMMPS commands.

    pair_coeff * * <list of model symbols>

__Format-specific build_potential_LAMMPS() parameters__

- __kimid__ (*str*) The unique kim ID for the potential.  Note that this is used rather than pair_style_terms.

In [6]:
b = potentials.build_potential_LAMMPS(pair_style='kim', elements='Al', kimid='AKSDFO!')

pot = b.potential()
print(pot.pair_info())

mass 1 26.9815385

pair_style kim AKSDFO!
pair_coeff * * Al



### 1.7 Other potentials

The current builders capture ~95% of all LAMMPS pair_styles. For the remaining pair_styles

- If the pair_style's pair_coeffs follow one of the above formats but is not currently known to follow that format, the underlying builder can be called directly.

- If the pair_style's pair_coeffs do not follow any of the above formats, a new builder can be made and/or the underlying potential_LAMMPS data model can be built by hand. Contact [potentials@nist.gov](mailto:potentials@nist.gov) for help.

## 2. Saving user-defined potentials

The above section descriptions provide a simple means of generating PotentialLAMMPS objects for new potentials not in the system.  This section shows how the same builders can be used to create the underlying potential_LAMMPS data models that can be saved for use later.

### 2.1 Additional metadata

The builders also take additional parameters associated with metadata fields for identifying and classifying the LAMMPS potentials

- __id__ (*str, optional*) A human-readable identifier to name the LAMMPS potential implementation.  Basically, how you want to refer to the potential.

- __key__ (*str, optional*) A UUID4 code to uniquely identify the LAMMPS potential implementation.  If not specified, a new UUID4 code is automatically generated.

- __potid__ (*str, optional*) A human-readable identifier to refer to the conceptual potential model that the potential is based on.  This should be shared by alternate implementations of the same potential.

- __potkey__ (*str, optional*) A UUID4 code to uniquely identify the conceptual potential model. This should be shared by alternate implementations of the same potential. If not specified, a new UUID4 code is automatically generated.

- __status__ (*str, optional*) Indicates if the implementation is 'active' (valid and current), 'superceded' (valid, but better ones exist), or 'retracted' (invalid). Default value is 'active'.

In [14]:
# Start with the eam/alloy example again
b = potentials.build_potential_LAMMPS(pair_style='eam/alloy', elements='Al', paramfile='Al.eam.alloy')

# Show the generated data model
print(b.build().json(indent=4))

{
    "potential-LAMMPS": {
        "key": "7525db6c-f0b8-4b7f-ab65-7fe2e7888fcd", 
        "id": null, 
        "potential": {
            "key": "ab955185-7d50-42f5-a87e-4a6717110caa", 
            "id": null
        }, 
        "units": "metal", 
        "atom_style": "metal", 
        "atom": {
            "element": "Al"
        }, 
        "pair_style": {
            "type": "eam/alloy"
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "Al.eam.alloy"
                }, 
                {
                    "symbols": "True"
                }
            ]
        }
    }
}


Note that the two id fields have null values.  They can be set either during initialization or as attributes after initialization.

In [16]:
b.id = 'Al-parameterization-round12-v2'
b.potid = 'Al-parameterization-round12'
# Show the generated data model
print(b.build().json(indent=4))

{
    "potential-LAMMPS": {
        "key": "7525db6c-f0b8-4b7f-ab65-7fe2e7888fcd", 
        "id": "Al-parameterization-round12-v2", 
        "potential": {
            "key": "ab955185-7d50-42f5-a87e-4a6717110caa", 
            "id": "Al-parameterization-round12"
        }, 
        "units": "metal", 
        "atom_style": "metal", 
        "atom": {
            "element": "Al"
        }, 
        "pair_style": {
            "type": "eam/alloy"
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "Al.eam.alloy"
                }, 
                {
                    "symbols": "True"
                }
            ]
        }
    }
}


### 2.2 Save locally

If the model content is saved locally, the potentials.Database class can load any records stored in that directory in addition to the downloaded records from the remote database.

Built-in function for saving locally comming soon...